# Using Zestful's free daily allowance to parse ingredients

In [1]:
import pandas as pd

In [24]:
posts = pd.read_csv("posts_preprocessed.csv")
posts_sample = pd.read_csv("posts_with_ingredients.csv")

posts['ingredient_lines'] = posts['ingredient_lines'].apply(eval)
posts_sample['ingredient_lines'] = posts_sample['ingredient_lines'].apply(eval)
posts_sample['ingredients_dict'] = posts_sample['ingredients_dict'].apply(eval)

In [50]:
# Already used posts 0,1,2,3,23,24,27,
# Time: 13:30
posts_new_sample = posts.iloc[[23,24,27]]

In [51]:
num_lines = 0
for lines in posts_new_sample['ingredient_lines']:
    print(f"Numer of lines {len(lines)}")
    num_lines += len(lines)
print(f"Total number of lines: {num_lines}")

Numer of lines 9
Numer of lines 9
Numer of lines 10
Total number of lines: 28


In [52]:
for lines in posts_new_sample['ingredient_lines']:
    print(lines)

['- 1.5 cup oats', '- 2 scoops unflavoured or vanilla protein powder', '- 1/2 teaspoon cinnamon', '- 1.5 -2 cup milk of choice (i use ) ', '- 2/3 cup greek yoghurt', '- 2 tablespoon chia seeds', '- maple syrup to taste (i used about 1 tablespoon)', '- 1 teaspoon vanilla essence (omit if using vanilla protein)', '- 1/4 cup dark chocolate chips']
['2 aubergines ', '2 minced garlic clove ', '2 tablespoon lemon juice ', '1/4 cup tahini ', '1/3 cup olive oil ', '2 tablespoon chopped flat leaf parsley', '1/4 teaspoon cumin ', 'salt n peps ', 'sprinkle of paprika ']
['1.5 cup iceberg lettuce', '3 tablespoon mayonnaise', '2 tablespoon red wine vinegar', '1/4 cup banana peppers', 'banana pepper juice (optional)', 'oregano (to taste)', '1 clove minced garlic', 'red pepper flakes (to taste)', 'salt & pepper (to taste)', '1/2 cup sliced red onion']


In [53]:
# zestful ingredient parser
import parse_ingredient

def get_ingredients_dict(ingredient_lines):
    
    ingredients = parse_ingredient.parse_multiple(ingredient_lines)
    
    return ingredients.as_dict()

In [55]:
posts_new_sample['ingredients_dict'] = posts_new_sample['ingredient_lines'].apply(get_ingredients_dict)

/var/folders/hn/dknbybgs0qv4myljwr8qtf4c0000gn/T/ipykernel_85412/3922682051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_new_sample['ingredients_dict'] = posts_new_sample['ingredient_lines'].apply(get_ingredients_dict)


In [57]:
for index, row in posts_new_sample.iterrows():
    print(row['ingredient_lines'])
    print(row['ingredients_dict'])
    print("\n-----------------------\n")

['- 1.5 cup oats', '- 2 scoops unflavoured or vanilla protein powder', '- 1/2 teaspoon cinnamon', '- 1.5 -2 cup milk of choice (i use ) ', '- 2/3 cup greek yoghurt', '- 2 tablespoon chia seeds', '- maple syrup to taste (i used about 1 tablespoon)', '- 1 teaspoon vanilla essence (omit if using vanilla protein)', '- 1/4 cup dark chocolate chips']
[{'error': None, 'raw': '- 1.5 cup oats', 'parsed': {'confidence': 0.9920374000000001, 'product': 'oats', 'productSizeModifier': None, 'quantity': 1.5, 'unit': 'cup', 'preparationNotes': None, 'usda_info': {'category': 'Breakfast Cereals', 'description': 'Cereals, oats, regular and quick, not fortified, dry', 'fdcId': '173904', 'matchMethod': 'exact'}}}, {'error': None, 'raw': '- 2 scoops unflavoured or vanilla protein powder', 'parsed': {'confidence': 0.413598, 'product': 'vanilla protein powder', 'productSizeModifier': None, 'quantity': 2.0, 'unit': 'scoop', 'preparationNotes': None, 'usda_info': None}}, {'error': None, 'raw': '- 1/2 teaspoon 

In [69]:
got_usda = 0
for ingredient_dict in posts_new_sample['ingredients_dict']:
    for ingredient in ingredient_dict:
        if ingredient['parsed']['usda_info']:
            got_usda += 1
print(f"Retrieved USDA info from {got_usda}/{num_lines} ingredients.")

Retrieved USDA info from 18/28 ingredients.


In [75]:
pd.concat([posts_sample,posts_new_sample]).to_csv("posts_with_ingredients.csv", index=False)

In [76]:
pd.read_csv("posts_with_ingredients.csv")

,Unnamed: 0,postUrl,profileUrl,username,fullName,commentCount,likeCount,pubDate,description,location,...,isSidecar,sidecarMedias,videoUrl,viewCount,language,score,description_preprocessed,includes_ingredient_list,ingredient_lines,ingredients_dict
0,81,https://www.instagram.com/p/Cgd_iR_vT2d/,https://www.instagram.com/all.about.oats,all.about.oats,Anushka Lodhi,0,2,2022-07-26T08:23:14.000Z,Chocolate fudge protein oatmeal🤎 💪\nIngredient...,"Ghaziabad, India",...,True,3.0,NaN,NaN,en,0.999997,chocolate fudge protein oatmeal \ningredients:...,True,"['1/2 cup instant oats', '1/2 cup milk', '1/2 ...","[{'error': None, 'raw': '1/2 cup instant oats'..."
1,104,https://www.instagram.com/p/Cgd8bLXDaCq/,https://www.instagram.com/a_m_eats,a_m_eats,Alice & Meg 🍴,3,12,2022-07-26T07:56:03.000Z,⁣Caprese Chicken with Pesto 🌿🍅⠀\n⠀\nThis dish ...,"Glasgow, United Kingdom",...,True,3.0,NaN,NaN,en,0.999997,⁣caprese chicken with pesto ⠀\n⠀\nthis dish is...,True,"['1 chicken breast⠀', '1 ball mozarella cheese...","[{'error': None, 'raw': '1 chicken breast⠀', '..."
2,169,https://www.instagram.com/p/CgdylqtOy9s/,https://www.instagram.com/hescottwellness,hescottwellness,"Natasha Hescott, RDN, CDN",0,1,2022-07-26T06:30:06.000Z,Looking for a fast recipe to make for lunch? T...,NaN,...,False,NaN,NaN,NaN,en,0.999994,looking for a fast recipe to make for lunch? t...,True,"['2 pound bok choy, small ', '1 tbs ginger, fr...","[{'error': None, 'raw': '2 pound bok choy, sma..."
3,172,https://www.instagram.com/p/CgdyENyLgmv/,https://www.instagram.com/rainbowpiatto,rainbowpiatto,Rainbow Piatto,1,14,2022-07-26T06:25:32.000Z,Sweet & Savoury Mushroom Scones~🍄 This easy an...,Singapore / Singapura / 新加坡 / சிங்கப்பூர்,...,False,NaN,NaN,NaN,en,0.999996,sweet & savoury mushroom scones~ this easy and...,True,"['200 gram swiss brown mushrooms, chopped ', '...","[{'error': None, 'raw': '200 gram swiss brown ..."
4,627,https://www.instagram.com/p/CgboCcgDiXs/,https://www.instagram.com/thealmostcook,thealmostcook,Mel 🍋,2,38,2022-07-25T10:19:26.000Z,overnight oats 🍯 if you’re wanting some easy a...,London Borough of Hackney,...,False,NaN,NaN,NaN,en,0.999996,overnight oats if you’re wanting some easy and...,True,"['- 1.5 cup oats', '- 2 scoops unflavoured or ...","[{'error': None, 'raw': '- 1.5 cup oats', 'par..."
5,635,https://www.instagram.com/p/Cgbk1gkj0Bj/,https://www.instagram.com/hangryinlondon,hangryinlondon,Hangry in London 🌶,0,16,2022-07-25T09:51:27.000Z,Finally got round to making baba ganoush! Not ...,Cornwall,...,True,2.0,NaN,NaN,en,0.999998,finally got round to making baba ganoush! not ...,True,"['2 aubergines ', '2 minced garlic clove ', '2...","[{'error': None, 'raw': '2 aubergines ', 'pars..."
6,781,https://www.instagram.com/p/Cgdw3BDKgw2/,https://www.instagram.com/ket_ofoodrecipesandtips,ket_ofoodrecipesandtips,Keto Food Recipes and Tips,1,5,2022-07-26T06:18:07.000Z,👉Save this post for the recipe.\nFollow \n@ket...,NaN,...,False,NaN,https://scontent-lhr8-2.cdninstagram.com/v/t50...,6.0,en,0.999996,save this post for the recipe.\nfollow \n to ...,True,"['1.5 cup iceberg lettuce', '3 tablespoon mayo...","[{'error': None, 'raw': '1.5 cup iceberg lettu..."
